# Trying Ckip Connection Model

давать награду за любой цвет

In [26]:
%load_ext autoreload
%autoreload 2

from functools import partial
from collections import defaultdict
import pickle

from wordle.wordlenp import Wordle
from environment.environment import Environment, StateYesNo, StateVocabulary
from environment.action import ActionVocabulary, ActionLetters, ActionCombLetters
from dqn.agent import Agent
from dqn.train import Trainer
from replay_buffer.cpprb import PrioritizedReplayBuffer, ReplayBuffer

import matplotlib.pyplot as plt
import seaborn as sns
sns.set(style='whitegrid')
import torch
import numpy as np
np.random.seed(0)

"cuda:0" if torch.cuda.is_available() else "cpu"

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


'cuda:0'

## Data

In [27]:
answers = Wordle._load_vocabulary('wordle/answers.txt', astype=list)
guesses = Wordle._load_vocabulary('wordle/guesses.txt', astype=list)
wordle_list = guesses

in_answers = []
for i, word in enumerate(guesses):
  if word in answers:
    in_answers.append(guesses.index(word))

print(len(answers), len(guesses))

indices = np.arange(len(guesses))
np.random.shuffle(indices)

2321 12972


## Train Configuration

In [29]:
ohe1 = ActionCombLetters(vocabulary=wordle_list, k=1).ohe_matrix
ohe2 = ActionCombLetters(vocabulary=wordle_list, k=2).ohe_matrix
print(ohe1.shape, ohe2.shape)
rewards = {'B':0, 'Y':1, 'G':1, 'win':10, 'lose':-10, 'step':-5}
tasks_results = defaultdict(dict)

torch.Size([130, 12972]) torch.Size([4782, 12972])


In [30]:
def train(**kwargs):
    return Trainer.train_comb_letters(rewards, ohe1, ohe2, wordle_list, tasks_results, **kwargs)

In [31]:
def agent_path(nickname, t):
    return {
        'local': f'{nickname}/local-{t}.pth',
        'target': f'{nickname}/target-{t}.pth',
        'buffer': f'{nickname}/buffer-{t}.npz',
        'optimizer': f'{nickname}/optimizer-{t}.pth',
    }

In [32]:
def get_data(n_guesses, overfit):
    return Trainer.train_test_split(n_guesses, overfit, guesses, indices, in_answers)

## Experiments

In [34]:
n_guesses = 4000
data = get_data(n_guesses, overfit=True)

guesses: 4000
train answers: 678
test answers: 678 (overfit strategy)


In [12]:
train(
    data=data,
    
    n_batches=40000,
    n_batches_warm=10,
    
    eps_start=1,
    eps_end=0.01,
    eps_decay=0.99954,

    alpha=0,
    rb_size=int(1e5),
    method_name='skip',
    
    n_envs=8,
    k=1,
    optimize_interval=8,

    agent_path=None
)

WARM BATCHES:   0%|          | 0/10 [00:00<?, ?it/s]


Batch    0	Test Win Rate: 0.44%	Test Mean Steps: 3.67


TRAIN BATCHES:   0%|          | 0/40000 [00:00<?, ?it/s]


Batch 5000	Time: 106 s	Agent Eps: 0.10	Train Win Rate: 3.08%	Test Win Rate: 3.84%	Test Mean Steps: 4.31

Batch 10000	Time: 216 s	Agent Eps: 0.01	Train Win Rate: 12.36%	Test Win Rate: 14.18%	Test Mean Steps: 3.99

Batch 15000	Time: 322 s	Agent Eps: 0.01	Train Win Rate: 36.52%	Test Win Rate: 40.18%	Test Mean Steps: 4.01

Batch 20000	Time: 415 s	Agent Eps: 0.01	Train Win Rate: 71.92%	Test Win Rate: 76.66%	Test Mean Steps: 3.92

Batch 25000	Time: 499 s	Agent Eps: 0.01	Train Win Rate: 89.16%	Test Win Rate: 91.29%	Test Mean Steps: 3.83

Batch 30000	Time: 577 s	Agent Eps: 0.01	Train Win Rate: 94.44%	Test Win Rate: 94.53%	Test Mean Steps: 3.60

Batch 35000	Time: 649 s	Agent Eps: 0.01	Train Win Rate: 97.04%	Test Win Rate: 97.49%	Test Mean Steps: 3.53

Batch 40000	Time: 718 s	Agent Eps: 0.01	Train Win Rate: 97.90%	Test Win Rate: 97.64%	Test Mean Steps: 3.37


'skip-677-4000'

In [20]:
n_guesses = 8000
data = get_data(n_guesses, overfit=True)

guesses: 8000
train answers: 1423
test answers: 1423 (overfit strategy)


In [21]:
train(
    data=data,
    
    n_batches=40000,
    n_batches_warm=0,
    
    eps_start=0.01,
    eps_end=0.01,
    eps_decay=1,
    
    alpha=0.4,
    rb_size=int(1e6),
    method_name='skip2',

    n_envs=8,
    k=1, 
    optimize_interval=8,

    agent_path=agent_path('skip-677-4000', 4)
)

WARM BATCHES: 0it [00:00, ?it/s]


Batch    0	Test Win Rate: 60.79%	Test Mean Steps: 4.19


TRAIN BATCHES:   0%|          | 0/40000 [00:00<?, ?it/s]


Batch 5000	Time: 93 s	Agent Eps: 0.01	Train Win Rate: 73.42%	Test Win Rate: 72.03%	Test Mean Steps: 4.46

Batch 10000	Time: 190 s	Agent Eps: 0.01	Train Win Rate: 82.82%	Test Win Rate: 87.00%	Test Mean Steps: 4.10

Batch 15000	Time: 291 s	Agent Eps: 0.01	Train Win Rate: 85.42%	Test Win Rate: 85.03%	Test Mean Steps: 4.39

Batch 20000	Time: 394 s	Agent Eps: 0.01	Train Win Rate: 88.12%	Test Win Rate: 93.46%	Test Mean Steps: 4.01

Batch 25000	Time: 499 s	Agent Eps: 0.01	Train Win Rate: 92.38%	Test Win Rate: 94.24%	Test Mean Steps: 4.06

Batch 30000	Time: 606 s	Agent Eps: 0.01	Train Win Rate: 93.64%	Test Win Rate: 95.01%	Test Mean Steps: 3.99

Batch 35000	Time: 712 s	Agent Eps: 0.01	Train Win Rate: 93.36%	Test Win Rate: 91.78%	Test Mean Steps: 4.25

Batch 40000	Time: 816 s	Agent Eps: 0.01	Train Win Rate: 94.26%	Test Win Rate: 96.77%	Test Mean Steps: 3.91


'skip2-1423-8000'

In [26]:
n_guesses = len(guesses)
data = get_data(n_guesses, overfit=True)

guesses: 12972
train answers: 2315
test answers: 2315 (overfit strategy)


In [28]:
train(
    data=data,
    
    n_batches=40000,
    n_batches_warm=0,
    
    eps_start=0.01,
    eps_end=0.01,
    eps_decay=1,
    
    alpha=0.4,
    rb_size=int(1e6),
    method_name='skip2',

    n_envs=8,
    k=1, 
    optimize_interval=8,

    agent_path=agent_path('skip2-1423-8000', 7)
)

WARM BATCHES: 0it [00:00, ?it/s]


Batch    0	Test Win Rate: 79.18%	Test Mean Steps: 4.26


TRAIN BATCHES:   0%|          | 0/40000 [00:00<?, ?it/s]


Batch 5000	Time: 93 s	Agent Eps: 0.01	Train Win Rate: 77.28%	Test Win Rate: 82.20%	Test Mean Steps: 4.20

Batch 10000	Time: 210 s	Agent Eps: 0.01	Train Win Rate: 82.42%	Test Win Rate: 72.22%	Test Mean Steps: 4.87

Batch 15000	Time: 329 s	Agent Eps: 0.01	Train Win Rate: 82.98%	Test Win Rate: 87.86%	Test Mean Steps: 4.22

Batch 20000	Time: 444 s	Agent Eps: 0.01	Train Win Rate: 89.84%	Test Win Rate: 89.94%	Test Mean Steps: 4.18

Batch 25000	Time: 556 s	Agent Eps: 0.01	Train Win Rate: 91.28%	Test Win Rate: 92.05%	Test Mean Steps: 4.15

Batch 30000	Time: 668 s	Agent Eps: 0.01	Train Win Rate: 91.72%	Test Win Rate: 92.22%	Test Mean Steps: 4.18

Batch 35000	Time: 778 s	Agent Eps: 0.01	Train Win Rate: 92.52%	Test Win Rate: 92.40%	Test Mean Steps: 4.17

Batch 40000	Time: 889 s	Agent Eps: 0.01	Train Win Rate: 92.80%	Test Win Rate: 94.08%	Test Mean Steps: 4.12


'skip2-2315-12972'

In [32]:
train(
    data=data,
    
    n_batches=40000,
    n_batches_warm=0,
    
    eps_start=0,
    eps_end=0,
    eps_decay=1,
    
    alpha=1,
    rb_size=int(1e6),
    method_name='skip2',
    lr=1e-5,
    fine_tune=True,

    n_envs=8,
    k=1,
    optimize_interval=8,

    agent_path=agent_path('skip2-2315-12972', 7)
)

WARM BATCHES: 0it [00:00, ?it/s]


Batch    0	Test Win Rate: 94.08%	Test Mean Steps: 4.12


TRAIN BATCHES:   0%|          | 0/40000 [00:00<?, ?it/s]


Batch 5000	Time: 80 s	Agent Eps: 0.00	Train Win Rate: 96.00%	Test Win Rate: 70.93%	Test Mean Steps: 4.95

Batch 10000	Time: 185 s	Agent Eps: 0.00	Train Win Rate: 96.20%	Test Win Rate: 96.76%	Test Mean Steps: 3.98

Batch 15000	Time: 291 s	Agent Eps: 0.00	Train Win Rate: 97.00%	Test Win Rate: 97.24%	Test Mean Steps: 3.99

Batch 20000	Time: 392 s	Agent Eps: 0.00	Train Win Rate: 97.12%	Test Win Rate: 97.19%	Test Mean Steps: 4.00

Batch 25000	Time: 491 s	Agent Eps: 0.00	Train Win Rate: 96.70%	Test Win Rate: 97.62%	Test Mean Steps: 3.91

Batch 30000	Time: 589 s	Agent Eps: 0.00	Train Win Rate: 98.34%	Test Win Rate: 98.14%	Test Mean Steps: 3.87

Batch 35000	Time: 686 s	Agent Eps: 0.00	Train Win Rate: 98.22%	Test Win Rate: 98.23%	Test Mean Steps: 3.86

Batch 40000	Time: 782 s	Agent Eps: 0.00	Train Win Rate: 98.78%	Test Win Rate: 99.18%	Test Mean Steps: 3.85


'skip2-2315-22972-2'

In [34]:
train(
    data=data,
    
    n_batches=40000,
    n_batches_warm=0,
    
    eps_start=0,
    eps_end=0,
    eps_decay=1,
    
    alpha=1,
    rb_size=int(1e6),
    method_name='skip2',
    lr=1e-5,
    fine_tune=True,

    n_envs=8,
    k=1,
    optimize_interval=8,

    agent_path=agent_path('skip2-2315-22972-2', 7)
)

WARM BATCHES: 0it [00:00, ?it/s]


Batch    0	Test Win Rate: 99.18%	Test Mean Steps: 3.85


TRAIN BATCHES:   0%|          | 0/40000 [00:00<?, ?it/s]


Batch 5000	Time: 72 s	Agent Eps: 0.00	Train Win Rate: 98.84%	Test Win Rate: 98.92%	Test Mean Steps: 3.81

Batch 10000	Time: 169 s	Agent Eps: 0.00	Train Win Rate: 98.86%	Test Win Rate: 98.66%	Test Mean Steps: 3.85

Batch 15000	Time: 265 s	Agent Eps: 0.00	Train Win Rate: 98.60%	Test Win Rate: 98.66%	Test Mean Steps: 3.82

Batch 20000	Time: 360 s	Agent Eps: 0.00	Train Win Rate: 98.76%	Test Win Rate: 99.05%	Test Mean Steps: 3.79

Batch 25000	Time: 455 s	Agent Eps: 0.00	Train Win Rate: 99.38%	Test Win Rate: 99.31%	Test Mean Steps: 3.78

Batch 30000	Time: 550 s	Agent Eps: 0.00	Train Win Rate: 98.74%	Test Win Rate: 99.09%	Test Mean Steps: 3.80

Batch 35000	Time: 645 s	Agent Eps: 0.00	Train Win Rate: 99.38%	Test Win Rate: 99.48%	Test Mean Steps: 3.78

Batch 40000	Time: 739 s	Agent Eps: 0.00	Train Win Rate: 99.36%	Test Win Rate: 99.09%	Test Mean Steps: 3.79


'skip2-3315-32972-3'

In [36]:
train(
    data=data,
    
    n_batches=40000,
    n_batches_warm=0,
    
    eps_start=0,
    eps_end=0,
    eps_decay=1,
    
    alpha=1,
    rb_size=int(1e6),
    method_name='skip2',
    lr=1e-6,
    fine_tune=True,

    n_envs=8,
    k=1,
    optimize_interval=8,

    agent_path=agent_path('skip2-3315-32972-3', 6)
)

WARM BATCHES: 0it [00:00, ?it/s]


Batch    0	Test Win Rate: 99.48%	Test Mean Steps: 3.78


TRAIN BATCHES:   0%|          | 0/40000 [00:00<?, ?it/s]


Batch 5000	Time: 71 s	Agent Eps: 0.00	Train Win Rate: 99.62%	Test Win Rate: 99.40%	Test Mean Steps: 3.79

Batch 10000	Time: 165 s	Agent Eps: 0.00	Train Win Rate: 97.88%	Test Win Rate: 99.52%	Test Mean Steps: 3.76

Batch 15000	Time: 257 s	Agent Eps: 0.00	Train Win Rate: 99.46%	Test Win Rate: 99.14%	Test Mean Steps: 3.76

Batch 20000	Time: 353 s	Agent Eps: 0.00	Train Win Rate: 99.36%	Test Win Rate: 98.88%	Test Mean Steps: 3.78

Batch 25000	Time: 448 s	Agent Eps: 0.00	Train Win Rate: 99.38%	Test Win Rate: 99.61%	Test Mean Steps: 3.75

Batch 30000	Time: 542 s	Agent Eps: 0.00	Train Win Rate: 99.64%	Test Win Rate: 99.18%	Test Mean Steps: 3.74

Batch 35000	Time: 636 s	Agent Eps: 0.00	Train Win Rate: 99.52%	Test Win Rate: 99.61%	Test Mean Steps: 3.73

Batch 40000	Time: 729 s	Agent Eps: 0.00	Train Win Rate: 99.66%	Test Win Rate: 99.74%	Test Mean Steps: 3.73


'skip2-4315-42972-4'

In [8]:
train(
    data=data,
    
    n_batches=160000,
    n_batches_warm=0,
    
    eps_start=0,
    eps_end=0,
    eps_decay=1,
    
    alpha=1,
    rb_size=int(1e6),
    method_name='skip2',
    lr=1e-6,
    fine_tune=True,
    logging_interval=5000,

    n_envs=8,
    k=1,
    optimize_interval=8,

    agent_path=agent_path('skip2-4315-42972-4', 7)
)

WARM BATCHES: 0it [00:00, ?it/s]


Batch    0	Test Win Rate: 99.74%	Test Mean Steps: 3.73


TRAIN BATCHES:   0%|          | 0/160000 [00:00<?, ?it/s]


Batch 5000	Time: 72 s	Agent Eps: 0.00	Train Win Rate: 97.92%	Test Win Rate: 99.65%	Test Mean Steps: 3.73

Batch 10000	Time: 165 s	Agent Eps: 0.00	Train Win Rate: 99.56%	Test Win Rate: 99.65%	Test Mean Steps: 3.73

Batch 15000	Time: 259 s	Agent Eps: 0.00	Train Win Rate: 99.54%	Test Win Rate: 99.83%	Test Mean Steps: 3.73

Batch 20000	Time: 352 s	Agent Eps: 0.00	Train Win Rate: 99.62%	Test Win Rate: 99.57%	Test Mean Steps: 3.72

Batch 25000	Time: 446 s	Agent Eps: 0.00	Train Win Rate: 99.84%	Test Win Rate: 99.91%	Test Mean Steps: 3.72

Batch 30000	Time: 539 s	Agent Eps: 0.00	Train Win Rate: 99.56%	Test Win Rate: 99.61%	Test Mean Steps: 3.72

Batch 35000	Time: 632 s	Agent Eps: 0.00	Train Win Rate: 99.60%	Test Win Rate: 99.57%	Test Mean Steps: 3.71

Batch 40000	Time: 725 s	Agent Eps: 0.00	Train Win Rate: 99.64%	Test Win Rate: 99.78%	Test Mean Steps: 3.72

Batch 45000	Time: 819 s	Agent Eps: 0.00	Train Win Rate: 99.70%	Test Win Rate: 99.65%	Test Mean Steps: 3.72

Batch 50000	Time: 911 s	Agent

'skip2-5315-52972-5'

### balance test set

In [35]:
n_guesses = len(guesses)
data = get_data(n_guesses, overfit=True)

guesses: 12972
train answers: 2321
test answers: 2321 (overfit strategy)


In [36]:
train(
    data=data,
    
    n_batches=160000,
    n_batches_warm=0,
    
    eps_start=0,
    eps_end=0,
    eps_decay=1,
    
    alpha=1,
    rb_size=int(1e6),
    method_name='skip2',
    lr=1e-6,
    fine_tune=True,
    logging_interval=5000,

    n_envs=8,
    k=1,
    optimize_interval=8,

    agent_path={
        'local': 'some-results/local-99_96.pth',
        'target': 'some-results/local-99_96.pth',
        'optimizer': 'some-results/optimizer-99_96.pth',
        'buffer': 'some-results/buffer-99_96.npz',
    }
)

WARM BATCHES: 0it [00:00, ?it/s]


Batch    0	Test Win Rate: 99.70%	Test Mean Steps: 3.69


TRAIN BATCHES:   0%|          | 0/160000 [00:00<?, ?it/s]


Batch 5000	Time: 69 s	Agent Eps: 0.00	Train Win Rate: 99.44%	Test Win Rate: 99.57%	Test Mean Steps: 3.70

Batch 10000	Time: 160 s	Agent Eps: 0.00	Train Win Rate: 99.58%	Test Win Rate: 99.57%	Test Mean Steps: 3.70

Batch 15000	Time: 252 s	Agent Eps: 0.00	Train Win Rate: 99.52%	Test Win Rate: 99.57%	Test Mean Steps: 3.70

Batch 20000	Time: 343 s	Agent Eps: 0.00	Train Win Rate: 99.78%	Test Win Rate: 99.66%	Test Mean Steps: 3.70

Batch 25000	Time: 435 s	Agent Eps: 0.00	Train Win Rate: 99.20%	Test Win Rate: 97.37%	Test Mean Steps: 3.75


KeyboardInterrupt: 